I verified in different points in time, whether the number of features, stop words and stemmed could influence the performance of the classifiers.

I found that 400 features provided the best results in terms of F1-Macro score; stop words are not influential as well as stemming

In [ ]:

#testing a) more word features may improve performance; noisy
#testing b) no stopwords
#testomg c) stemmed word features; noisy vs preprocessed

#a) 

most_fq_words = nltk.FreqDist(w for w in content_text)
word_features_experiment = list(most_fq_words)
quantity_words = [100, 150, 200, 300, 400, 410, 420, 450, 500]

results_morefeat_NB = []
results_morefeat_LOG = []

for number in quantity_words:
    more_word_features = list(word_features_experiment)[:number]
    
    train_more = [(document_features(word_tokenize(w.lower(), language='danish'),word_features), label) for w, label in set_training]
    val_more = [(document_features(word_tokenize(w.lower(), language='danish'),word_features), label) for w, label in set_validation]
    test_more = [(document_features(word_tokenize(w.lower(), language='danish'),word_features), label) for w, label in set_test]
    
    clfNB_more = nltk.NaiveBayesClassifier.train(train_more)
    pred_NB_more = [clfNB_more.classify(p) for p, _ in val_set]
    
    clfLOG_more = SklearnClassifier(LogisticRegression(random_state = 10,multi_class='ovr',
                                                        penalty='l2',# class_weight= weights_cl
                                                        class_weight=weights_manual)).train(train_set)
    pred_LOG_more = [clfLOG_more.classify(p) for p, _ in val_set]
    
    y_val = [(y) for _, y in val_set]
    f1_NB_more = f1_score(y_val, pred_NB_more, average='macro')
    f1_LOG_more = f1_score(y_val, pred_LOG_more, average='macro')

    results_morefeat_NB.append([number, f1_NB_more])
    results_morefeat_LOG.append([number, f1_LOG_more])
    
results_morefeat_NB = pd.DataFrame(results_morefeat_NB, columns = ["Number of most occurring words-features", "F1-Score of Naive Bayes"])
results_morefeat_LOG = pd.DataFrame(results_morefeat_LOG, columns = ["Number of most occurring words-features", "F1-Score of Logistic"])
results_morefeat_NB 
results_morefeat_LOG


# b) Naive Bayes

from nltk.corpus import stopwords
dansk_stopwords = set(stopwords.words('danish'))
content_no_stopwords = []
for name in content:
    name = name.lower()
    # name = name.replace(".", "")
    name = word_tokenize(name, language='danish')
    for token in name:
        if token != "":
            if token not in dansk_stopwords:
                content_no_stopwords.append(token)
content_no_stopwords[:10]

without_stopwords = nltk.FreqDist(t.lower() for t in content_no_stopwords)
n_words = [100, 150, 200, 300, 400, 410, 420, 450, 500]

results_no_stopwords = []
results_nostpw_log = []
n_words = [100, 150, 200, 300, 400, 410, 420, 450, 500]
for number in n_words:
    nostpw_features = list(without_stopwords)[:number]
    
    train_nostpw = [(document_features(word_tokenize(w.lower(), language='danish'),nostpw_features), label) for w, label in set_training]
    val_nostpw = [(document_features(word_tokenize(w.lower(), language='danish'),nostpw_features), label) for w, label in set_validation]
    test_nostpw = [(document_features(word_tokenize(w.lower(), language='danish'),nostpw_features), label) for w, label in set_test]
    
    clfNB_nostpw = nltk.NaiveBayesClassifier.train(train_nostpw)
    pred_nostpw = [clfNB_nostpw.classify(p) for p, _ in val_nostpw]
    clfNB_nostop_fscore = f1_score(y_val, pred_nostpw, average='macro')
    
    log_clf_nostpw = SklearnClassifier(LogisticRegression(random_state = 10,multi_class='ovr',penalty='l2',class_weight=weights_manual)).train(train_nostpw)
    pred_log_nostpw = [log_clf_nostpw.classify(p) for p, _ in val_nostpw]
    log_nostpw_f1 = f1_score(y_val, pred_log_nostpw, average='macro')
    
    results_nostpw_log.append([number, log_nostpw_f1])
    results_no_stopwords.append([number, clfNB_nostop_fscore])

results_nostpw_log = pd.DataFrame(results_nostpw_log, columns = ["Number words-features no stopwords", "F1-Score of LOG"])
results_no_stopwords = pd.DataFrame(results_no_stopwords, columns = ["Number words-features no stopwords", "F1-Score of classifier"])
results_no_stopwords
results_nostpw_log



# c) 

stemmer = SnowballStemmer('danish')
stemmed = [stemmer.stem(word) for word in content_text]
stemmed_words = nltk.FreqDist(w.lower() for w in stemmed)

results_NB_stemmed = []
results_LOG_stemmed = []
n_words = [100, 150, 200, 300, 400, 410, 420, 450, 500]
for number in n_words:
    words_features_stemmed = list(stemmed_words)[:number]
    
    train_stemmed = [(document_features(word_tokenize(w.lower(), language='danish'),words_features_stemmed), label) for w, label in set_training]
    val_stemmed = [(document_features(word_tokenize(w.lower(), language='danish'),words_features_stemmed), label) for w, label in set_validation]
    test_stemmed = [(document_features(word_tokenize(w.lower(), language='danish'),words_features_stemmed), label) for w, label in set_test]
    
    clfNB_stemmed = nltk.NaiveBayesClassifier.train(train_stemmed)
    pred_NB_stemmed = [clfNB_stemmed.classify(p) for p, _ in val_stemmed]
    clfNB_stemmed_fscore = f1_score(y_val, pred_NB_stemmed, average='macro')
    
    clfLog_stemmed = SklearnClassifier(LogisticRegression(random_state = 10,multi_class='ovr',penalty='l2',class_weight=weights_manual)).train(train_stemmed)
    pred_log_stemmed = [clfLog_stemmed.classify(p) for p, _ in val_stemmed]
    log_stemmed_f1 = f1_score(y_val, pred_log_stemmed, average='macro')

    results_NB_stemmed.append([number, clfNB_stemmed_fscore])
    results_LOG_stemmed.append([number, log_stemmed_f1])

results_NB_stemmed = pd.DataFrame(results_NB_stemmed, columns = ["Number stemmed words-features", "F1-Score of NB"])
results_LOG_stemmed = pd.DataFrame(results_LOG_stemmed, columns = ["Number stemmed words-features", "F1-Score of LOG"])
results_NB_stemmed
results_LOG_stemmed